In [1]:
import json
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
import shutil

In [2]:
# Load label data from imprint_labels_vetted.json
with open('labels/imprint_labels_vetted.json') as f:
    label_data = json.load(f)

# Create label mapping
image_to_labels = {item['image']: item['labels'] for item in label_data}
all_chars = sorted(set(label['label'] for item in label_data for label in item['labels']))
char_to_idx = {char: idx for idx, char in enumerate(all_chars)}
print(f"Classes: {len(all_chars)} characters")
print(f"Total labeled images: {len(label_data)}")

Classes: 64 characters
Total labeled images: 773


In [3]:
# Setup YOLO dataset structure - clean first to avoid stale files
base_dir = Path('yolo_dataset')

# Remove existing dataset directory if it exists
if base_dir.exists():
    shutil.rmtree(base_dir)
    print("Removed existing yolo_dataset directory")

# Create fresh directory structure
for split in ['train', 'val']:
    (base_dir / split / 'images').mkdir(parents=True, exist_ok=True)
    (base_dir / split / 'labels').mkdir(parents=True, exist_ok=True)

# Split data - use only images from imprint_labels_vetted.json
image_names = [item['image'] for item in label_data]
train_imgs, val_imgs = train_test_split(image_names, test_size=0.15, random_state=42)
print(f"Train: {len(train_imgs)}, Val: {len(val_imgs)}")

Removed existing yolo_dataset directory
Train: 657, Val: 116


In [4]:
# Convert to YOLO format and copy images
image_source = Path('../data/pillbox_production_images_full_202008')

for img_name, split in [(img, 'train') for img in train_imgs] + [(img, 'val') for img in val_imgs]:
    src = image_source / img_name
    if not src.exists():
        continue
    
    # Copy image
    dst_img = base_dir / split / 'images' / img_name
    shutil.copy(src, dst_img)
    
    # Create YOLO label file
    labels = image_to_labels[img_name]
    label_path = base_dir / split / 'labels' / f"{Path(img_name).stem}.txt"
    
    with open(label_path, 'w') as f:
        for label in labels:
            cls = char_to_idx[label['label']]
            coords = label['coords'].split()
            x, y, w, h = map(float, coords)
            # Ensure coords are in valid range [0, 1]
            x, y, w, h = max(0, min(1, x)), max(0, min(1, y)), max(0, min(1, w)), max(0, min(1, h))
            f.write(f"{cls} {x} {y} {w} {h}\n")

print("Dataset prepared")

Dataset prepared


In [5]:
# Create YOLO config file
import yaml

config = {
    'path': str(base_dir.absolute()),
    'train': 'train/images',
    'val': 'val/images',
    'nc': len(all_chars),
    'names': all_chars
}

with open(base_dir / 'data.yaml', 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

print("Config created")

Config created


In [8]:
# Train YOLO model
model = YOLO('yolov8n.pt')
results = model.train(
    data=str(base_dir / 'data.yaml'),
    epochs=50,
    imgsz=640,
    batch=16,
    name='pill_imprint_v16'  # Full training run with 773 vetted images
)

Ultralytics 8.3.233 🚀 Python-3.13.2 torch-2.9.1 CPU (Apple M1)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=pill_imprint_v16, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=

In [9]:
# Load auto_label.csv and predict
auto_df = pd.read_csv('auto_label.csv')
trained_model = YOLO('../runs/detect/pill_imprint_v16/weights/best.pt')

predictions = []
for _, row in auto_df.iterrows():
    img_path = image_source / row['original_name']
    if not img_path.exists():
        continue
    
    results = trained_model(img_path, verbose=False)
    for result in results:
        boxes = result.boxes
        for box in boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            x, y, w, h = box.xywhn[0].tolist()
            predictions.append({
                'image': row['original_name'],
                'label': all_chars[cls],
                'coords': f"{x} {y} {w} {h}",
                'confidence': conf
            })

pred_df = pd.DataFrame(predictions)
print(f"Generated {len(pred_df)} predictions")

Generated 40432 predictions


In [10]:
# Group by image and save to JSON format
auto_labeled = []
for img_name in pred_df['image'].unique():
    img_preds = pred_df[pred_df['image'] == img_name]
    auto_labeled.append({
        'image': img_name,
        'labels': [
            {'label': row['label'], 'coords': row['coords'], 'confidence': row['confidence']}
            for _, row in img_preds.iterrows()
        ]
    })

with open('labels/auto_labeled_predictions.json', 'w') as f:
    json.dump(auto_labeled, f, indent=2)

# Show confidence distribution
print(f"\\nConfidence stats:")
print(pred_df['confidence'].describe())
print(f"\\nHigh confidence (>0.8): {len(pred_df[pred_df['confidence'] > 0.8])}")
print(f"Medium confidence (0.5-0.8): {len(pred_df[(pred_df['confidence'] > 0.5) & (pred_df['confidence'] <= 0.8)])}")
print(f"Low confidence (<0.5): {len(pred_df[pred_df['confidence'] <= 0.5])}")

\nConfidence stats:
count    40432.000000
mean         0.586930
std          0.234877
min          0.250030
25%          0.360977
50%          0.567483
75%          0.808132
max          0.997768
Name: confidence, dtype: float64
\nHigh confidence (>0.8): 10492
Medium confidence (0.5-0.8): 12435
Low confidence (<0.5): 17505


# EDA on predicted labels

In [4]:
# EDA: Analyze confidence distribution by image
import json

# Load the auto-labeled predictions
with open('labels/auto_labeled_predictions.json') as f:
    auto_predictions = json.load(f)

# Calculate average confidence per image
image_confidences = []
for item in auto_predictions:
    if len(item['labels']) > 0:
        avg_conf = sum(label['confidence'] for label in item['labels']) / len(item['labels'])
        image_confidences.append({
            'image': item['image'],
            'avg_confidence': avg_conf,
            'num_labels': len(item['labels'])
        })

# Create bins
bins = [
    ('>90%', 0.90, 1.0),
    ('>80%', 0.80, 0.90),
    ('>70%', 0.70, 0.80),
    ('>60%', 0.60, 0.70),
    ('>50%', 0.50, 0.60),
    ('<50%', 0.0, 0.50)
]

print("=" * 90)
print("CONFIDENCE DISTRIBUTION BY IMAGE (Average Confidence)")
print("=" * 90)
print(f"Total images with predictions: {len(image_confidences)}\n")

cumulative_count = 0
cumulative_percent = 0.0

for bin_name, min_conf, max_conf in bins:
    if bin_name == '<50%':
        count = sum(1 for img in image_confidences if img['avg_confidence'] < max_conf)
    else:
        count = sum(1 for img in image_confidences if min_conf <= img['avg_confidence'] < max_conf)
    
    percentage = (count / len(image_confidences) * 100) if len(image_confidences) > 0 else 0
    cumulative_count += count
    cumulative_percent += percentage
    
    # Create output with cumulative stats
    print(f"{bin_name:>6}: {count:4d} images ({percentage:5.1f}%)  |  Cumulative: {cumulative_count:4d} ({cumulative_percent:5.1f}%)")

print("=" * 90)

# Show some statistics
conf_values = [img['avg_confidence'] for img in image_confidences]
print(f"\nAverage confidence across all images: {sum(conf_values) / len(conf_values):.3f}")
print(f"Median confidence: {sorted(conf_values)[len(conf_values)//2]:.3f}")
print(f"Min confidence: {min(conf_values):.3f}")
print(f"Max confidence: {max(conf_values):.3f}")

CONFIDENCE DISTRIBUTION BY IMAGE (Average Confidence)
Total images with predictions: 6449

  >90%:  113 images (  1.8%)  |  Cumulative:  113 (  1.8%)
  >80%:  539 images (  8.4%)  |  Cumulative:  652 ( 10.1%)
  >70%: 1023 images ( 15.9%)  |  Cumulative: 1675 ( 26.0%)
  >60%: 1435 images ( 22.3%)  |  Cumulative: 3110 ( 48.2%)
  >50%: 1542 images ( 23.9%)  |  Cumulative: 4652 ( 72.1%)
  <50%: 1797 images ( 27.9%)  |  Cumulative: 6449 (100.0%)

Average confidence across all images: 0.596
Median confidence: 0.593
Min confidence: 0.252
Max confidence: 0.984


In [5]:
# Filter images that have average confidence of more than 75%
high_confidence_threshold = 0.75

# Filter images above threshold
high_confidence_images = [img['image'] for img in image_confidences if img['avg_confidence'] > high_confidence_threshold]

# Create new JSON with only high confidence predictions
high_confidence_data = []
for item in auto_predictions:
    if item['image'] in high_confidence_images:
        high_confidence_data.append(item)

# Save to new JSON file
output_path = 'labels/high_confidence_auto_labels_rd1.json'
with open(output_path, 'w') as f:
    json.dump(high_confidence_data, f, indent=2)

print(f"Filtered {len(high_confidence_data)} images with average confidence > {high_confidence_threshold*100}%")
print(f"Saved to: {output_path}")

# Show some stats about the filtered data
total_labels = sum(len(item['labels']) for item in high_confidence_data)
print(f"\nTotal character labels in high confidence set: {total_labels}")
print(f"Average labels per image: {total_labels / len(high_confidence_data):.1f}")

Filtered 1090 images with average confidence > 75.0%
Saved to: labels/high_confidence_auto_labels_rd1.json

Total character labels in high confidence set: 5863
Average labels per image: 5.4
